Imports and more 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import psycopg2 as psycopg2
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.metrics import *
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix 
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
import time

Retive Data for Test and Training. The data is put into two sets at present 

In [ ]:
startTid = time.time()

In [ ]:
#def getDataSets(number):
    # Connect to an existing database
conn = None
try:
    conn = psycopg2.connect(#database="public",
                        port = "5432",
                        host="localhost", 
                        user="postgres", 
                        password="1234")
except psycopg2.DatabaseError as e: 
        print (f'Error {e}')
        sys.exit(1)
            
    # Open a cursor to perform database operations
cur = conn.cursor()

    # Execute a command: this creates a new table

cur.execute(("""SELECT article.type, article.content, url.sld
from article 
inner join url 
on article.id = url.article_id 
and article.type IN ('fake','satire','bias','conspiracy','junksci','clickbait','political','reliable','state');"""))
# cur.execute(("""SELECT article.type, article.content, author.name  
# from article 
# inner join author 
# on article.id = author.id 
# and article.type IN ('fake','satire','bias','conspiracy','junksci','clickbait','political','reliable','state');"""))
df = (cur.fetchmany(1000000))

    # Make the changes to the database persistent
conn.commit()

    # Close communication with the database
cur.close()
conn.close()


Naive Bayes classifier for multinomial models.

The multinomial Naive Bayes classifier is suitable for classification with discrete features (e.g., word counts for text classification). The multinomial distribution normally requires integer feature counts. However, in practice, fractional counts such as tf-idf may also work.

Tager fra SKLearns hjemmeside :) 

In [ ]:
df_articles = pd.DataFrame(df, columns =["type", "content", "url"])
#df_articles = pd.DataFrame(df, columns =["type", "content", "author"])

In [ ]:
# Sorting our y variables into FAKE or REAL
df_articles["type"] = np.where(df_articles["type"] == "hate", "FAKE", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "fake", "FAKE", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "unreliable", "FAKE", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "conspiracy", "FAKE", df_articles["type"])

df_articles["type"] = np.where(df_articles["type"] == "satire", "FAKE", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "junksci", "FAKE", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "bias", "FAKE", df_articles["type"])

df_articles["type"] = np.where(df_articles["type"] == "clickbait", "REAL", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "political", "REAL", df_articles["type"])

df_articles["type"] = np.where(df_articles["type"] == "reliable", "REAL", df_articles["type"])
df_articles["type"] = np.where(df_articles["type"] == "state", "REAL", df_articles["type"])

df_articles["type"] = np.where(df_articles["type"] != "FAKE", "REAL", df_articles["type"])

In [ ]:
print(df_articles.shape)

In [ ]:
stemmer = PorterStemmer()
words = stopwords.words("english")
t1 = time.time()
Data_transformed = df_articles['content'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
t2 = time.time()
kage = t2-t1
print(kage)

In [ ]:
# Creating a vectorized array with integers indicating frequency of words.
vectorize = CountVectorizer(max_features=200,ngram_range=(1,3))
x = vectorize.fit_transform(Data_transformed).toarray()
y = np.array(df_articles['type'])

In [ ]:
# Creating a test split of 80/20.
xtrain, xtest, ytrain, ytest = train_test_split(x,y,test_size=0.2,random_state=34)
print('\nTotal number of samples in the Train Dataset :',xtrain.shape[0])
print('Total number of samples in the Test Dataset :',xtest.shape[0])

In [ ]:
# Second baseline model using multinomial naive bayes.
mulNB = MultinomialNB().fit(xtrain, ytrain)
print('\nAccuracy score for Multinomial Naive Bayes :',mulNB.score(xtest,ytest))

ypred_mul = mulNB.predict(xtest)

print('Confusion Metrics for Multinomial Naive Bayes : \n\n',confusion_matrix(ytest, ypred_mul),'\n\n')
print('Classification Report for Multinomial Naive Bayes :\n\n',classification_report(ytest,ypred_mul))

In [ ]:
# Third baseline model using Logistic regression.
scaler = StandardScaler()
lr = LogisticRegression(max_iter = 600)
log_model = Pipeline([('standardize', scaler),
                    ('log_reg', lr)])

log_model.fit(xtrain, ytrain)

ypred_log = log_model.predict(xtest)
test_accuracy = accuracy_score(ytest, ypred_log)*100

print('Testing accuracy for Logistic regression: %.4f %%' % test_accuracy) 
print('Confusion Metrics for Logistic regression:\n\n', confusion_matrix(ytest, ypred_log))
print('Classification Report for Multinomial Naive Bayes :\n\n',classification_report(ytest,ypred_log))

In [ ]:
slutTid = time.time()
kage2 = slutTid-startTid
print(kage2)
print("Tid i min:" ,kage2/60)